# Pandas | Core concepts, types and methods (part II)

Szymon Talaga 10.01.2020

<hr>

In this notebook we continue our quest for developing solid, in-depth understanding of Pandas data structures and their relations to Numpy.

Here we will focus on the most important type which is `DataFrame`. In the later part we will also consider `MultiIndex` type
and advanced methods for indexing hierarchical data structures. In the first part of the notebook we will focus on flat indexes
with only one level (i.e. standard `Index` objects).

Data frame in Pandas is a two dimensional collection of values arranged as a rectangular grid of rows and columns.
It is primarily oriented column-wise as the columns are represented as `Series` objects. This means that any single column of a `DataFrame` has to be
of a fixed `dtype`, but different columns may have different `dtypes`. This is a primary reason why (usually) row-wise operations in Pandas
are less efficient and in general more difficult to carry out.

Data frames use also a more complex indexing architecture. All `Series` defining columns have to share the same index. This allows to have a well-defined
row index. Additionally, a data frame has also a horizontal index which defines columns (and possibly also groups of columns etc.).

Summing up, data frames can be viewed as a collection of columns represented as `Series` which are mapped to column names organized as an `Index` 
(or `MultiIndex`) object combined with another `Index` (or `MultiIndex`) object that define row indexes and which is shared by all column `Series`.

Although the abovementioned view is not 100% veridical with respect to the true internal data model used by Pandas it rather captures the general
design of Pandas data frames. It also correctly points to the strong and weak points of Pandas data model in terms of what kinds of operations
are easy and hard to perform (in terms of computational efficiency).

```python
=====================================================
|       | column 1    column 2   . . .     column m |
=====================================================
| row 1 |    x           x                    x     |
| row 2 |    x           x                    x     |
|   .   |                                           |
|   .   |                                           |
|   .   |                                           |
| row n |    x           x                    x     |
=====================================================
```

<hr>

Many great resources about Pandas can be found in the official documentation. In particular, it is recommended to read the following articles:

* [10 minutes to Pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html)
* [Essential basic functionality](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html)
* [Intro to data structures](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html)
* [Indexing and selecting data](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-attribute-access)

<hr>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# Helper package to load example datasets
import seaborn as sbn

## `DataFrame` | 2D rectangular dataset organized as a collection of named `Series`

In this section we will review most important ways in which one can initialize a `DataFrame` object. 
This will provide not only a practical exercise but also important insights into the ways in which we can think about data frames.

Perhaps the most natural, although not necessarily the best, way to think about a data frame is to think about it
in terms of a list of rows defining a rectangular table.

In this spirit, we can build a data frame from a list of lists defining rows.

In [ ]:
pd.DataFrame([
    [1, 2, 3],
    [4, 5, 6, 4],
    [7, 8, 9],
    [10, 11, 12]
])

This way we will not get any column or row names by default. However, this can be changed by using `columns` and `index` arugments.

Note that we do not have to pass type homogeneous data. Below we create a data frame of which third column is of `object` type
(it contains strings) while the first two ones are integer.

In [ ]:
df = pd.DataFrame([
    [1, 2, 'a string 1'],
    [4, 5, 'a string 2'],
    [7, 8, 'a string 3'],
    [10, 11, 'a string 4']
], columns=['a', 'b', 'c'], index=['p', 'r', 's', 't'])
df

We can check types using `.dtypes` attribute of a data frame.

In [ ]:
df.dtypes

Similarly we can look at row and column indexes.

In [ ]:
print(df.index)    # row index
print(df.columns)  # column index

A similar approach is to create a data frame from a sequence of dictionaries. This way we can specify column names in data.
However, if we want to have non-generic row labels we still need to provide them separately through `index` argument.

NOTE. The ordering of columns is defined by the ordering of keys in the first dictionary.

In [ ]:
pd.DataFrame([
    {'b': 'string', 'a': 2, 'c': 11111},
    {'a': 1, 'b': 'another string'},
], index=['s1', 's2'])

We can fully define data and both indexes in a row-wise fashion if we define a data frame based on a list of named `Series`
which are interpreted as rows in this case.

In [ ]:
pd.DataFrame([
    pd.Series({'a': 1, 'b': 2}, name='subject 1'),
    pd.Series({'a': 2, 'b': 3}, name='subject 2')
])

From the column perspective we can define a data frame as a named collection (a mapping) of columns.

In [ ]:
pd.DataFrame({
    'a': [1, 2, 3],
    'b': [4, 5, 6],
    'c': ['x', 'y', 'z']
}, index=['s1', 's2', 's3'])

In order to see that pandes really enforces equal length of columns as well as identical indexes, we can check what happens
if try to create data frame from a collection non-conformable series.

In [ ]:
# Unequal lengths
pd.DataFrame({
    'a': [1, 2, 3, 999],
    'b': [4, 5, 6],
    'c': ['x', 'y', 'z']
})

When we pass series with different indexes we do not get an error. Instead we obtain a data frame partially filled with NaNs,
which is of course the result of the rules of labels alignment in Pandas.

In [ ]:
# Unequal indexes
pd.DataFrame({
    'a': pd.Series([1, 2, 3]),
    'b': pd.Series([4, 5, 6]),
    'c': pd.Series(['x', 'y', 'z'], index=['uu', 'vv', 'ww'])
})

# `DataFrame` | Indexing and slicing

`DataFrame` objects provide the same three syntaxes and three kinds of indexing. The main difference is that data frames
have two separate axes (rows and columns).

In [ ]:
df = pd.DataFrame([
    pd.Series({'a': 1, 'b': 2, 'c': 'foo'}, name='s1'),
    pd.Series({'a': 11, 'b': 22, 'c': 'bar'}, name='s2'),
    pd.Series({'a': 20, 'b': 30, 'c': 'xoxo'}, name='s3'),
    pd.Series({'a': 7, 'b': 15, 'c': 'yoyo'}, name='s4'),
    pd.Series({'a': 50, 'b': 1, 'c': 'howdy'}, name='s5')
])
df

### _getitem_ syntax

It can be used to select columns (single or multiple).

In [ ]:
# Get single column
# Output is a series
df['b']

In [ ]:
# Get multiple columns
# Output is a data frame
df[['a', 'c']]

In [ ]:
# Get single column as a data frame
df[['b']]

However, if we provide a slice in _getitem_ indexing it will be interpreted as slice over row labels
or positions if it is an integer slice. Thus, in this case we have a similar problem with ambiguity of the _getitem_ syntax
as in the case of `Series`.

In [ ]:
# Get slice of rows by label
df['s2':'s4']

In [ ]:
# Get slice of rows by integer positions
df[1:4]

In [ ]:
# Ambiguous case of data frame with integer labels
df2 = df.set_index(pd.Index([2, 3, 1, 4, 0]))
df2

In [ ]:
# Get rows by integer slice
# They are interpreted as positions and not labels
df2[2:4]

### `DataFrame` | `.loc` indexing

It provides label-based indexing for both rows and columns.

In [ ]:
df

In [ ]:
# Get row (as a Series)
df.loc['s2', :]

In [ ]:
# Get column (as a Series)
df.loc[:, 'b']

In [ ]:
# Get column (as a DataFrame)
df.loc[:, ['b']]

In [ ]:
# Get multiple columns and rows
df.loc[['s2', 's4'], ['a', 'c']]

In [ ]:
# Get slices of rows and columns
df.loc['s2':, :'b']

### `DataFrame` | `.iloc` indexing

It provides position-based indexing for both rows and columns.

In [ ]:
df

In [ ]:
# Single element
df.iloc[2, 2]

In [ ]:
# Single row (as Series)
df.iloc[-1, :]

In [ ]:
# Single row (as DataFrame)
df.iloc[[-1], :]

In [ ]:
# Multiple rows and columns
df.iloc[[1, -1], [0, -1]]

In [ ]:
# Slices
df.iloc[:3, 1:]

### `DataFrame` | Boolean indexing

As in the case of `Series` we can provide 1D boolean masks to filer rows and/or columns.
What is important is the fact that with data frames we may mix boolean masks with other types of indexing.
For instance, we can have a boolean mask on columns and label-based or positional index on rows.

In [ ]:
df

In [ ]:
df.columns
df.columns.isin(['a', 'c'])

In [ ]:
df.iloc[2:, df.columns.isin(['a', 'c'])]

In [ ]:
df['a'] > 10

In [ ]:
df.loc[df['a'] > 10, ['c', 'a']]

A new feature is that we can also provide a full boolean mask over entire data frame to mask particular values
abd turn them into NaNs.

In [ ]:
X = np.random.uniform(-1, 1, (10, 5, 3, 2))
X[X > 0].shape

In [ ]:
np.random.seed(101010)

num = pd.DataFrame(np.random.normal(0, 1, (10, 3)), columns=['x', 'y', 'z'])
num

In [ ]:
num >= 0

In [ ]:
# Mask negative values
num[num >= 0].sum(0, skipna=False)

## `DataFrame` | Basic attributes and descriptions

In [ ]:
## .columns
## Column index
df.columns

In [ ]:
## .index
## Row index
df.index

In [ ]:
## .dtypes
## Column names and their dtypes
df.dtypes

In [ ]:
## len()
## Number of rows
len(df)

In [ ]:
## .shape
## Tuple with number of rows and columns
df.shape

In [ ]:
## .info()
## Show basic information about a data frame
df.info()

In [ ]:
## .describe()
## Basic numeric summary of data
## Categorical columns are ommited by default if there are any numeric columns
df.describe()

In [ ]:
df['c'].describe()

In [ ]:
## .to_numpy()
## Numpy representation
df.to_numpy()

In [ ]:
df.head()
df.tail()

## `DataFrame` | Broadcasting and labels alignment

Here we review the broadcasting and labels alignment rules for data frames.
In general they are the same as for `Series` objects. However, in this case we have two axes instead
of one, so this induces some additional complications.

The main rule is that we do labels alignment for both row and column indexes.

In [ ]:
# Two simple data frames
df1 = pd.DataFrame(np.arange(4).reshape(2, 2), columns=['a', 'b'], index=[3, 7])
df1

In [ ]:
df2 = pd.DataFrame(np.arange(4, 8).reshape(2, 2), columns=['b', 'a'], index=[7, 3])
df2

In [ ]:
# Add them together
df2 + df1

In [ ]:
# What happened step by step
row_union = df1.index.union(df2.index)
row_union

In [ ]:
col_union = df1.columns.union(df2.columns)
col_union

In [ ]:
#df1 = df1.reindex(index=row_union).reindex(columns=col_union)
df1 = df1.reindex(index=row_union).reindex(columns=col_union)
df1

In [ ]:
df2 = df2.reindex(index=row_union).reindex(columns=col_union)
df2

In [ ]:
# Final operation
df1 + df2

Now let us see what happens if axes can not be perfecly aligned (they are at least partially non-overlapping).

In [ ]:
df1 = pd.DataFrame(np.arange(8).reshape(4, 2), index=[2, 4, 6, 8], columns=['a', 'b'])
df1

In [ ]:
df2 = pd.DataFrame(np.arange(8).reshape(4, 2), index=[2, 3, 4, 5], columns=['b', 'c'])
df2

In [ ]:
df1 + df2

In [ ]:
# What happened step by step
row_union = df1.index.union(df2.index)
row_union

In [ ]:
col_union = df1.columns.union(df2.columns)
col_union

In [ ]:
df1 = df1.reindex(index=row_union, columns=col_union)
df1

In [ ]:
df2 = df2.reindex(index=row_union, columns=col_union)
df2

In [ ]:
# Final operation
df1 + df2

But how are aligned operations between series and data frames? The rule is that index labels of a series are matched
with column labels of a data frame. It makes it easy to define operations based on columns and their aggregate
values such as centering.

In [ ]:
# Center columns of numeric data frame
df = pd.DataFrame(np.random.normal(100, 15, (10, 3)), columns=['x', 'y', 'z'])
df

In [ ]:
# Column means
# Mean computed over columns
df.mean(axis=0)

In [ ]:
df_c = df - df.mean(axis=0)

In [ ]:
# Check if it is really centered
df_c.mean(0)

Is it as easy carry out operations row-wise? Nope. But it can be done, although it requires some additional tricks.
However, first let us see and understand what is going on, when we try to broadcast between data frame and 
a series representing row-aggregated values (i.e. row centering).

In [ ]:
df

In [ ]:
# row means
df.mean(axis=1)

In [ ]:
# Remove means from rows
df_c = df - df.mean(axis=1)
df_c

Total disaster! Do you understand what happened?

One way, although a little convoluted and not really the best one, to deal with this is to use transposition.
Data frames are inherently two-dimensional (they have rows and columns) like matrices. So we can easily
define a transpose of a data frame. Below is the transpose of our original data frame.

In [ ]:
df.T

With this trick in our hands now we can reexpress our row-oriented problem as column oriented problem
and convert the result back to the original orientation with yet another transpose.

In [ ]:
df.T.mean(0)

In [ ]:
df_c = (df.T - df.mean(1)).T
df_c

In [ ]:
# Check that it worked
df_c.mean(1)

In [ ]:
df = pd.DataFrame({
    'x': [1, 2, 3],
    'y': [1., 2., 3.]
})
df.dtypes

In [ ]:
df.T.T.dtypes

### `DataFrame` | Exercise 1.

You are provided with a simple numeric data frame. Standardize it both column and row-wise.
In other words both column and row means should be $0$ and standard deviations (and variances) should be $1$.

Remember that the formula for standardization is the following:

$$X_{\text{standardized}} = \frac{X - \text{Mean}(X)}{\text{Std}(X)}$$

In [ ]:
np.random.seed(101)

df = pd.DataFrame(np.random.normal(100, 15, (10, 3)), columns=['x', 'y', 'z'])
df

In [ ]:
# Your solution

# Columnwise
df_c = ((df - df.mean(0)) / df.std(0))
df_c

In [ ]:
df_r = df.T
df_r = (df_r - df_r.mean(0)) / df_r.std(0)
df_r = df_r.T

df_r = ((df.T - df.mean(1)) / df.std(1)).T
        
df_r.mean(1)
df_r.std(1)

## Flexible operations (arithmetic, logical etc.)

We managed to implement row-wise operation with some smart use of transposition. However, this seems rather hacky
and we would like to have some better tools for doing just that. Happily Pandas provides us with such tools.

`DataFrame` and `Series` objects in Pandas implements special methods called _flexible operations_ which are
just standard arithmetic and logical operation, but such that can be explicitly applied along a given axis.
Moreover, they can also automatically substitute NaNs which are created during labels alignment.

**Flexible arithmetic binary operations**

1. `add`
2. `sub`
3. `div`
4. `mul`
5. `pow`

**Flexible logical binary operations**

1. `eq` (equal)
2. `ne` (not equal)
3. `lt` (lower than)
4. `gt` (greater than)
5. `le` (lower or equal)
6. `ge` (greater or equal)

In [ ]:
df = pd.DataFrame(np.random.normal(100, 15, (10, 2)), columns=['x', 'y'])
df

In [ ]:
df - df.mean(0)

In [ ]:
df.sub(df.mean(0), axis=1)

In [ ]:
df.sub(df.mean(1), axis=0)

In [ ]:
# Adding partially matching series with automatic substitution of NaNs
s1 = pd.Series([1, 2, np.nan], index=['a', 'b', 'c'])
s1

In [ ]:
s2 = pd.Series([6, 4, 7, 8], index=['a', 'd', 'f', 'b'])
s2

In [ ]:
s1 + s2

In [ ]:
index_union = s1.index.union(s2.index)
s1.reindex(index_union).fillna(0) + s2.reindex(index_union).fillna(0)

In [ ]:
s1.add(s2, fill_value=0)

In [ ]:
# What happened step by step
# Step 1. Index union.
index_union = s1.index.union(s2.index)
index_union

In [ ]:
# Step 2. Reindex series
s1 = s1.reindex(index_union)
s1

In [ ]:
s2 = s2.reindex(index_union)
s2

In [ ]:
# Step 3. Check where both series have NaNs
nan_both = s1.isna() & s2.isna()
nan_both

In [ ]:
# Step 4. Fill NaNs where only one series have missing data
s1[~nan_both] = s1[~nan_both].fillna(0)
s1

In [ ]:
s2[~nan_both] = s2[~nan_both].fillna(0)
s2

In [ ]:
# Step 5. Carry out the operation
s1 + s2

### `DataFrame` | Exercise 2.

You are provided with simple numeric data frame (again!). Normalize it by rows and column with the Min-Max scaling.
The lowest value should be $0$ and highest should be $1$.

$$\frac{X - \text{Min}(X)}{\text{Max}(X) - \text{Min}(X)}$$

Do it separately two times. Once normalize columns and then normalize rows. Do not normalize both rows and columns
at the same time as with Min-Max scaling such operation does not make any sense! If you are curious you
can do this and check how the data frame looks like in this case.

In [ ]:
np.random.seed(101)

df = pd.DataFrame(np.random.normal(100, 15, (10, 3)), columns=['x', 'y', 'z'])
df

In [ ]:
# Your solution

df_c = (df - df.min(0)) / (df.max(0) - df.min(0))

# Columns
df_c = df.sub(df.min(0), axis=1).div(df.max(0) - df.min(0), axis=1)
df_c.min(0)
df_c.max(0)

# Rows
df_c = df.sub(df.min(1), axis=0).div(df.max(1) - df.min(1), axis=0)
df_c.min(1)
df_c.max(1)

### `DataFrame` | Exercise 3.

You are provided with a list of subject ids and two sets of measurements for those subjects from an experiment
with two trials. However, some subjects may have participated in only one or even none of the trials.

Your task is to compute average scores. For subjects with only one score the available score should be presented.
Subjects with no data should be assigned with $-999$.

HINT. You may want to use `.reindex()` method.

In [2]:
np.random.seed(303)

subj = pd.Index(range(30)) # List of subject ids
x1 = pd.Series(np.random.normal(90, 10, (25,)), index=np.random.choice(subj, size=(25,), replace=False))
x2 = pd.Series(np.random.normal(100, 15, (20,)), index=np.random.choice(subj, size=(20,), replace=False))

In [8]:
# Solution one
index_intersection = x1.index.intersection(x2.index)
index_intersection

Int64Index([26, 12, 4, 5, 25, 14, 21, 8, 28, 9, 20, 0, 1, 6, 17, 18], dtype='int64')

In [10]:
x1 = x1.reindex(subj)
x2 = x2.reindex(subj)
x1.size
x2.size

30

30

In [11]:
sums = x1.add(x2, fill_value=0)
sums

0     221.652299
1     182.768420
2      84.990160
3      93.407508
4     178.775451
5     186.499329
6     171.873288
7      80.915094
8     181.998174
9     176.121783
10     91.999743
11           NaN
12    185.222161
13     81.584400
14    185.908081
15     64.362858
16     92.615038
17    182.076839
18    209.408118
19     96.786868
20    182.116109
21    188.068242
22     87.684813
23     78.804619
24    117.747469
25    208.276604
26    199.923466
27     77.451740
28    183.711559
29    109.880340
dtype: float64

In [13]:
# Divide by 2 values where two measurements were available
sums[index_intersection] /= 2
sums = sums.fillna(-999)
sums

0     110.826149
1      91.384210
2      84.990160
3      93.407508
4      89.387726
5      93.249664
6      85.936644
7      80.915094
8      90.999087
9      88.060891
10     91.999743
11   -999.000000
12     92.611081
13     81.584400
14     92.954041
15     64.362858
16     92.615038
17     91.038419
18    104.704059
19     96.786868
20     91.058055
21     94.034121
22     87.684813
23     78.804619
24    117.747469
25    104.138302
26     99.961733
27     77.451740
28     91.855780
29    109.880340
dtype: float64

In [18]:
# Solution two (simpler one)
pd.DataFrame({
    'x1': x1.reindex(subj),
    'x2': x2.reindex(subj)
}).mean(axis=1).fillna(-999)

0     110.826149
1      91.384210
2      84.990160
3      93.407508
4      89.387726
5      93.249664
6      85.936644
7      80.915094
8      90.999087
9      88.060891
10     91.999743
11   -999.000000
12     92.611081
13     81.584400
14     92.954041
15     64.362858
16     92.615038
17     91.038419
18    104.704059
19     96.786868
20     91.058055
21     94.034121
22     87.684813
23     78.804619
24    117.747469
25    104.138302
26     99.961733
27     77.451740
28     91.855780
29    109.880340
dtype: float64

## `DataFrame` | Column and row-wise operations aka _apply_

We can do a lot with vectorization and labels alignment. However, sometimes we may want to apply arbitrary
functions to columns or rows. For this we can use `.apply()` method.

Apply is a sort of map-like statement in which one specifies a function that will be applied to every item
(in this context items are rows or columns). Below is a simple example.

In [21]:
np.random.seed(101)

df = pd.DataFrame(np.random.normal(100, 15, (10, 3)), columns=['x', 'y', 'z'])
df

,x,y,z
0,140.602748,109.421991,113.619542
1,107.557386,109.766769,95.210229
2,87.278845,109.089480,69.727476
3,111.101831,107.932202,91.164992
4,102.830430,88.616919,86.001442
5,114.325848,102.861915,129.681360
6,139.089509,110.252633,104.539982
7,125.405844,74.408711,82.613209
8,97.977389,105.857918,102.503570
9,102.767528,112.115589,101.094395


In [20]:
np.log(df)

,x,y,z
0,4.945939,4.695212,4.732856
1,4.678025,4.698358,4.556087
2,4.469108,4.692168,4.244594
3,4.710447,4.681503,4.512671
4,4.633081,4.484323,4.454364
5,4.739053,4.633387,4.865080
6,4.935118,4.702774,4.649570
7,4.831555,4.309573,4.414170
8,4.584737,4.662098,4.629898
9,4.632469,4.719530,4.616055


In [22]:
# Apply logarithm to columns
df.apply(np.log)

,x,y,z
0,4.945939,4.695212,4.732856
1,4.678025,4.698358,4.556087
2,4.469108,4.692168,4.244594
3,4.710447,4.681503,4.512671
4,4.633081,4.484323,4.454364
5,4.739053,4.633387,4.865080
6,4.935118,4.702774,4.649570
7,4.831555,4.309573,4.414170
8,4.584737,4.662098,4.629898
9,4.632469,4.719530,4.616055


In [23]:
len(df)

10

In [26]:
df.apply(len, axis=1)

0    3
1    3
2    3
3    3
4    3
5    3
6    3
7    3
8    3
9    3
dtype: int64

In [ ]:
df.sum()

In [ ]:
# Apply sum to columns
df.apply(np.sum)

In [28]:
# Apply as column filter
df.apply(lambda x: x[x > 100])

,x,y,z
0,140.602748,109.421991,113.619542
1,107.557386,109.766769,NaN
2,NaN,109.089480,NaN
3,111.101831,107.932202,NaN
4,102.830430,NaN,NaN
5,114.325848,102.861915,129.681360
6,139.089509,110.252633,104.539982
7,125.405844,NaN,NaN
8,NaN,105.857918,102.503570
9,102.767528,112.115589,101.094395


In [29]:
df[df > 100]

,x,y,z
0,140.602748,109.421991,113.619542
1,107.557386,109.766769,NaN
2,NaN,109.089480,NaN
3,111.101831,107.932202,NaN
4,102.830430,NaN,NaN
5,114.325848,102.861915,129.681360
6,139.089509,110.252633,104.539982
7,125.405844,NaN,NaN
8,NaN,105.857918,102.503570
9,102.767528,112.115589,101.094395


In [31]:
# Non-trivial apply
# Interquartile range per column
df.apply(lambda x: x.quantile(.75) - x.quantile(.25))

x    19.852591
y     6.069659
z    16.738549
dtype: float64

We can also apply function to rows. To do that we use additional `axis` argument.

In [32]:
# Compute row ranges
df.apply(lambda x: x.max() - x.min(), axis=1)

0    31.180757
1    14.556540
2    39.362004
3    19.936839
4    16.828988
5    26.819445
6    34.549527
7    50.997133
8     7.880528
9    11.021194
dtype: float64

When we work with `.apply()` on columns the situation is simple as we can expect that our function will be applied to
`Series` objects which are guaranteed to have fixed data types, so operations should be rather efficient.

On the hand, it is not entirely clear what the representation of rows should be as they may contain values of
different type. In general rows will also be represented as `Series` objects just such that are upcasted to an
appropriate `dtype` that can store all the values. However, this means that row-wise apply will be very often
much slower.

We can see representation of a single item during apply by simply passing a function that prints items.

In [33]:
# Representation of columns
_ = df.apply(print)

0    140.602748
1    107.557386
2     87.278845
3    111.101831
4    102.830430
5    114.325848
6    139.089509
7    125.405844
8     97.977389
9    102.767528
Name: x, dtype: float64
0    109.421991
1    109.766769
2    109.089480
3    107.932202
4     88.616919
5    102.861915
6    110.252633
7     74.408711
8    105.857918
9    112.115589
Name: y, dtype: float64
0    113.619542
1     95.210229
2     69.727476
3     91.164992
4     86.001442
5    129.681360
6    104.539982
7     82.613209
8    102.503570
9    101.094395
Name: z, dtype: float64


In [34]:
# Representation of rows
_ = df.apply(print, axis=1)

x    140.602748
y    109.421991
z    113.619542
Name: 0, dtype: float64
x    107.557386
y    109.766769
z     95.210229
Name: 1, dtype: float64
x     87.278845
y    109.089480
z     69.727476
Name: 2, dtype: float64
x    111.101831
y    107.932202
z     91.164992
Name: 3, dtype: float64
x    102.830430
y     88.616919
z     86.001442
Name: 4, dtype: float64
x    114.325848
y    102.861915
z    129.681360
Name: 5, dtype: float64
x    139.089509
y    110.252633
z    104.539982
Name: 6, dtype: float64
x    125.405844
y     74.408711
z     82.613209
Name: 7, dtype: float64
x     97.977389
y    105.857918
z    102.503570
Name: 8, dtype: float64
x    102.767528
y    112.115589
z    101.094395
Name: 9, dtype: float64


In both cases these are series with nice fixed dtype `float64`. But this is so only because the data frame we use
is simple and contain only floating point numbers. See what happens when we really have mixed types.

In [36]:
import seaborn as sbn

iris = sbn.load_dataset('iris')

iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [37]:
# Print row items
_ = iris.head().apply(print, axis=1)

sepal_length       5.1
sepal_width        3.5
petal_length       1.4
petal_width        0.2
species         setosa
Name: 0, dtype: object
sepal_length       4.9
sepal_width          3
petal_length       1.4
petal_width        0.2
species         setosa
Name: 1, dtype: object
sepal_length       4.7
sepal_width        3.2
petal_length       1.3
petal_width        0.2
species         setosa
Name: 2, dtype: object
sepal_length       4.6
sepal_width        3.1
petal_length       1.5
petal_width        0.2
species         setosa
Name: 3, dtype: object
sepal_length         5
sepal_width        3.6
petal_length       1.4
petal_width        0.2
species         setosa
Name: 4, dtype: object


We are forced to work with `object` series. This will usually negatively impact efficiency of our computations.
That is the reason why row apply is often slower.

If `Series` are returned by function used in apply they will be combined to form a data frame.

Below we show this with a non-trivial function that computes series with different fields for rows
of the iris dataset based on the value of species.

In [38]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [39]:
def row_func(row):
    if row['species'] == 'setosa':
        return pd.Series({'sepal': row['sepal_length'] / row['sepal_width']})
    return pd.Series({'petal': row['petal_length'] / row['petal_width']})

iris.apply(row_func, axis=1)

,petal,sepal
0,NaN,1.457143
1,NaN,1.633333
2,NaN,1.468750
3,NaN,1.483871
4,NaN,1.388889
...,...,...
145,2.260870,NaN
146,2.631579,NaN
147,2.600000,NaN
148,2.347826,NaN


Data frames also define `.applymap()` method that applies a function element-wise.

Below we use the method to find all the prime numbers in a data frame with non-negative integers.

CAUTION. The implementation of the test for primality we use here is SUPER BAD.

In [40]:
def is_prime(x):
    if x < 2:
        return False
    for i in range(2, x):
        if x % i == 0:
            return False
    return True

idf = pd.DataFrame(np.random.randint(0, 100, (10, 20)))
idf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,97,93,24,36,63,19,35,30,10,60,20,27,8,86,26,87,46,47,54,86
1,9,45,2,18,58,92,11,10,94,35,28,3,83,84,47,14,69,60,69,51
2,6,88,71,68,23,35,79,98,67,82,57,77,46,3,46,29,86,21,21,81
3,23,94,71,20,27,75,5,49,86,89,63,82,77,3,56,14,49,87,52,13
4,47,49,24,20,64,52,60,47,29,60,53,11,40,91,45,97,24,36,38,9
5,52,67,43,1,79,68,68,61,18,51,14,28,17,87,46,52,16,70,71,84
6,10,62,96,57,23,86,85,26,76,66,54,17,65,57,89,2,80,50,66,88
7,79,93,6,92,42,22,20,25,97,54,71,72,80,93,64,63,80,38,45,35
8,25,95,75,72,11,76,79,50,22,59,66,1,34,37,57,35,42,44,49,31
9,79,85,3,55,73,93,94,99,40,54,88,94,86,17,68,17,18,60,83,82


In [42]:
idf.applymap(is_prime)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False
1,False,False,True,False,False,False,True,False,False,False,False,True,True,False,True,False,False,False,False,False
2,False,False,True,False,True,False,True,False,True,False,False,False,False,True,False,True,False,False,False,False
3,True,False,True,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True
4,True,False,False,False,False,False,False,True,True,False,True,True,False,False,False,True,False,False,False,False
5,False,True,True,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False
6,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False
7,True,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False
8,False,False,False,False,True,False,True,False,False,True,False,False,False,True,False,False,False,False,False,True
9,True,False,True,False,True,False,False,False,False,False,False,False,False,True,False,True,False,False,True,False


In [44]:
idf[idf.applymap(is_prime)].fillna('')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,97,,,,,19,,,,,,,,,,,,47,,
1,,,2,,,,11,,,,,3,83,,47,,,,,
2,,,71,,23,,79,,67,,,,,3,,29,,,,
3,23,,71,,,,5,,,89,,,,3,,,,,,13
4,47,,,,,,,47,29,,53,11,,,,97,,,,
5,,67,43,,79,,,61,,,,,17,,,,,,71,
6,,,,,23,,,,,,,17,,,89,2,,,,
7,79,,,,,,,,97,,71,,,,,,,,,
8,,,,,11,,79,,,59,,,,37,,,,,,31
9,79,,3,,73,,,,,,,,,17,,17,,,83,


### `DataFrame` | Exercise 4.

You are provided with a data frame of exam scores of students. Each student have three exam scores ranging from 0 to 100.
You have to convert scores to grades according to the grading scale below and compute average grades of students.

**Grading scale**

If score is:

* $< 60 \rightarrow 2$
* $< 80 \rightarrow 3$
* $< 90 \rightarrow 4$
* $\geq 90 \rightarrow 5$

In [45]:
df = pd.DataFrame([
    pd.Series([61, 70, 90], name='Alice'),
    pd.Series([50, 80, 91], name='Bob'),
    pd.Series([80, 90, 82], name='Freya'),
    pd.Series([90, 100, 92], name='Merlin')
])
df

,0,1,2
Alice,61,70,90
Bob,50,80,91
Freya,80,90,82
Merlin,90,100,92


In [46]:
# Your solution
def score_to_grade(x):
    if x < 60:
        return 2
    if x < 80:
        return 3
    if x < 90:
        return 4
    return 5

df.applymap(score_to_grade).mean(axis=1)

Alice     3.666667
Bob       3.666667
Freya     4.333333
Merlin    5.000000
dtype: float64

### `DataFrame` | Exercise 5.

Your are provided with a set of exam scores for $10$ students (in a form of a `dict`).
Your task is to build a data frame in which row index values are student names,
the first column stores exam scores and the second column stores grade according to the following rule:

If score is:

* $< 60 \rightarrow 2$
* $< 80 \rightarrow 3$
* $< 90 \rightarrow 4$
* $\geq 90 \rightarrow 5$

HINT. You may want to convert the `dict` to a `Series` first. Then you can use `.apply()` or `.map()`
to apply some computations to every element of the series.

In [48]:
scores = {
    'Alice': 75,
    'Bob': 80,
    'Kate': 82,
    'Dog': 99,
    'Han Solo': 55,
    'Rick': 100,
    'Morty': 82,
    'Santa': 62,
    'Curie': 92,
    'Isabelle': 88,
    'Stan': 71,
    'Kyle': 81,
    'Kenny': 90,
    'Cartman': 30
}

In [54]:
dct

{'a': 1, 'b': 2, 'c': 111}

In [60]:
# Your solution
df = pd.DataFrame({
    'score': list(scores.values())
}, index=scores.keys())

df['grade'] = df['score'].apply(score_to_grade)
df

,score,grade
Alice,75,3
Bob,80,4
Kate,82,4
Dog,99,5
Han Solo,55,2
Rick,100,5
Morty,82,4
Santa,62,3
Curie,92,5
Isabelle,88,4


## `DataFrame` | Aggregation

Data frames in Pandas offer a quite convenient interface for computing multiple aggregate quantities in one go.

In [65]:
import numpy as np
import pandas as pd
import seaborn as sbn

iris = sbn.load_dataset('iris')

In [66]:
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [67]:
iris.loc[:, 'sepal_length':'petal_width'].agg([np.mean, np.std])

,sepal_length,sepal_width,petal_length,petal_width
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238


In [ ]:
iris['species'].nunique()

In [72]:
'nunique' in dir(iris['species'])

True

In [81]:
iris.aggregate({
    'sepal_length': [np.mean, np.std],
    'species': 'nunique'
})

,sepal_length,species
mean,5.843333,NaN
nunique,NaN,3.0
std,0.828066,NaN


## Split-apply-combine and `.groupby()`

Split-apply-combine is a powerful strategy used frequently in data science, statistics and scientific computing.
The main idea is that we can decompose even very complicate computations in a sequence of the three steps:

* **Split.** In this step we split our dataset into smaller datasets based on some criterion, for instance based
on value of some categorical variables.
* **Apply.** In this stage a function or a set of functions is applied to datasets. In this step we often aggregate
subdatasets into single values or perform some other processing (i.e. filtering).
* **Combine.** In the end we combine splitted datasets back to one data frame.

One of the typical usecases of this approach is computation of descriptive statistics for groups (i.e. group means).

Grouped operations in Pandas are defined via the `.groupby()` method. It returnes a special object
that stores a dataset parts divided by a given criterion (usually by values of a column or columns).
It can be used to iterate over the dataset parts but also to apply different functions to them.
After a function is applied the grouped object will return results combined back to a single data frame.
This is how _split-apply-combine_ strategy is implemented in Pandas.

In [2]:
iris = sbn.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
# Group iris data by species
iris_g = iris.groupby(['species'])
iris_g

In [4]:
# Iterate over groups and their names (criterion values)
for name, group in iris_g:
    print(name, "\n=========\n", group.head())

setosa 
    sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
versicolor 
     sepal_length  sepal_width  petal_length  petal_width     species
50           7.0          3.2           4.7          1.4  versicolor
51           6.4          3.2           4.5          1.5  versicolor
52           6.9          3.1           4.9          1.5  versicolor
53           5.5          2.3           4.0          1.3  versicolor
54           6.5          2.8           4.6          1.5  versicolor
virginica 
      sepal_length  sepal_width  petal_length  petal_width    species
100           6.3          3.3           6.0          2.5  virginica
101           5.8          2.7

In [7]:
# Compute group means
iris.groupby('species')[['sepal_length', 'sepal_width']].mean()

,sepal_length,sepal_width
species,,
setosa,5.006,3.428
versicolor,5.936,2.770
virginica,6.588,2.974


In [8]:
# Standard functions like mean can be used with even simpler syntax
iris.groupby('species').apply(np.mean)
iris.groupby('species').mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [9]:
df = pd.DataFrame(np.random.normal(100, 15, (10, 2)))
(df - df.mean()) / df.std()

,0,1
0,0.562355,1.282140
1,0.637496,0.420877
2,0.421389,0.069464
3,-1.731148,-0.741192
4,0.744750,1.247917
5,0.519337,0.779500
6,-0.728858,0.389885
7,-0.414017,-1.580326
8,1.331479,-1.114717
9,-1.342784,-0.753548


In [17]:
# We can also use apply and transform methods with groupyby
#
iris = sbn.load_dataset('iris')
# Below we standardize numeric variables in groups
z = iris \
    .groupby('species') \
    .apply(lambda x: (x - x.mean()) / x.std()) \
    .combine_first(iris)

z

,petal_length,petal_width,sepal_length,sepal_width,species
0,-0.357011,-0.436492,0.266674,0.189941,setosa
1,-0.357011,-0.436492,-0.300718,-1.129096,setosa
2,-0.932836,-0.436492,-0.868111,-0.601481,setosa
3,0.218813,-0.436492,-1.151807,-0.865288,setosa
4,-0.357011,-0.436492,-0.017022,0.453749,setosa
...,...,...,...,...,...
145,-0.637803,0.997633,0.176134,0.080621,virginica
146,-1.000191,-0.458766,-0.452916,-1.469783,virginica
147,-0.637803,-0.094666,-0.138391,0.080621,virginica
148,-0.275415,0.997633,-0.610178,1.320944,virginica


In [18]:
# Check solution
z.groupby('species').agg([np.mean, np.var])

petal_length        petal_width       sepal_length       \
                    mean  var          mean  var          mean  var   
species                                                               
setosa     -1.167955e-15  1.0  9.281464e-16  1.0  1.845399e-15  1.0   
versicolor  4.196643e-16  1.0  8.204548e-16  1.0  1.143530e-16  1.0   
virginica   6.528111e-16  1.0  6.417089e-16  1.0  2.748912e-15  1.0   

             sepal_width       
                    mean  var  
species                        
setosa     -2.167155e-15  1.0  
versicolor -1.486589e-15  1.0  
virginica   7.280287e-16  1.0

In [19]:
z.mean(0)
z.std(0)

petal_length   -3.108624e-17
petal_width     8.060219e-16
sepal_length    1.567635e-15
sepal_width    -9.770888e-16
dtype: float64

petal_length    0.993266
petal_width     0.993266
sepal_length    0.993266
sepal_width     0.993266
dtype: float64

In [21]:
# We may use just a subset of columns
Q = iris.groupby('species')['sepal_length'] \
    .apply(lambda x: x.quantile([0, .25, .5, .75, 1]))
Q

species         
setosa      0.00    4.300
            0.25    4.800
            0.50    5.000
            0.75    5.200
            1.00    5.800
versicolor  0.00    4.900
            0.25    5.600
            0.50    5.900
            0.75    6.300
            1.00    7.000
virginica   0.00    4.900
            0.25    6.225
            0.50    6.500
            0.75    6.900
            1.00    7.900
Name: sepal_length, dtype: float64

In [22]:
# And apply different functions to different columns
iris.groupby('species').agg({
    'sepal_length': [np.mean, np.std],
    'petal_length': [np.mean, np.var],
    'species': ['nunique']
})

sepal_length           petal_length           species
                   mean       std         mean       var nunique
species                                                         
setosa            5.006  0.352490        1.462  0.030159       1
versicolor        5.936  0.516171        4.260  0.220816       1
virginica         6.588  0.635880        5.552  0.304588       1

In [23]:
# Of course we can also ask for group sizes
iris.groupby('species').size()

species
setosa        50
versicolor    50
virginica     50
dtype: int64

All the methods above are useful but fall short when we need to perform complex computations depending on multiple
columns in groups. Luckily, there is a trick that we can use while using the `.apply()` method that allows us
to define arbitrary group computations using multiple columns as well as any other values we may need.

Below we compute average of ratios of sepal / petal lengths and widths in groups.

In [24]:
iris.groupby('species') \
    .apply(lambda g: pd.Series({
        'sepal': (g['sepal_length'] / g['sepal_width']).mean(),
        'petal': (g['petal_length'] / g['petal_width']).mean()
}))

,sepal,petal
species,,
setosa,1.470188,6.908000
versicolor,2.160402,3.242837
virginica,2.230453,2.780662


Of course it is also possible to group by multiple columns.

Below we show it using a famous dataset about Titanic survivors and casualties.

In [25]:
titanic = sbn.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [38]:
titanic.groupby(['class', 'embark_town']).size()

class   embark_town
First   Cherbourg       85
        Queenstown       2
        Southampton    127
Second  Cherbourg       17
        Queenstown       3
        Southampton    164
Third   Cherbourg       66
        Queenstown      72
        Southampton    353
dtype: int64

### `DataFrame` | Exercise 6.

Check how class and gender correlated with chance of survival in Titanic.

HINT. Use `groupby` (duh!)

HINT2. Note that the variable `survived` is simple binary vector, so you can compute its mean.

In [27]:
titanic = sbn.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Index hierarchies aka `MultiIndex`

So far we limited our attention to data structures with flat indexes that have only one level and (usually) map one unique
index value to one data value. But sometimes we may need something more elaborate. 

For instance, in one of the exercises we used a simple dataset with multiple exam scores for students.

In [39]:
df = pd.DataFrame([
    pd.Series([61, 70, 90], name='Alice'),
    pd.Series([50, 80, 91], name='Bob'),
    pd.Series([80, 90, 82], name='Freya'),
    pd.Series([90, 100, 92], name='Merlin')
])
df

,0,1,2
Alice,61,70,90
Bob,50,80,91
Freya,80,90,82
Merlin,90,100,92


In Pandas we may choose to represent the above data as a series object with two-level index that maps every data value
to student name and the exam number.

In [40]:
s = df.stack()
s

Alice   0     61
        1     70
        2     90
Bob     0     50
        1     80
        2     91
Freya   0     80
        1     90
        2     82
Merlin  0     90
        1    100
        2     92
dtype: int64

In [41]:
type(s)

pandas.core.series.Series

In [46]:
s.index

MultiIndex([( 'Alice', 0),
            ( 'Alice', 1),
            ( 'Alice', 2),
            (   'Bob', 0),
            (   'Bob', 1),
            (   'Bob', 2),
            ( 'Freya', 0),
            ( 'Freya', 1),
            ( 'Freya', 2),
            ('Merlin', 0),
            ('Merlin', 1),
            ('Merlin', 2)],
           )

Now we can use both index levels to address particular parts of our data.

In [43]:
s['Alice']

0    61
1    70
2    90
dtype: int64

In [47]:
s['Alice':'Freya']

Alice  0    61
       1    70
       2    90
Bob    0    50
       1    80
       2    91
Freya  0    80
       1    90
       2    82
dtype: int64

We can also use values on two-levels. We pass it as a tuple.

In [48]:
s[('Merlin', 2)]

92

Indexing on one of the nested level is a special kind of an operation that is called _cross-sectioning_. 
We have to use a special method to do this.

In [119]:
# Get scores for second test
s.xs(2, level=1)

Alice     90
Bob       91
Freya     82
Merlin    92
dtype: int64

We can also easily convert back and forth between multi index representation and simpler data frame representation.

In [49]:
s

Alice   0     61
        1     70
        2     90
Bob     0     50
        1     80
        2     91
Freya   0     80
        1     90
        2     82
Merlin  0     90
        1    100
        2     92
dtype: int64

In [52]:
df = s.reset_index().rename(columns={'level_0': 'name', 'level_1': 'exam', 0: 'score'})
df

,name,exam,score
0,Alice,0,61
1,Alice,1,70
2,Alice,2,90
3,Bob,0,50
4,Bob,1,80
5,Bob,2,91
6,Freya,0,80
7,Freya,1,90
8,Freya,2,82
9,Merlin,0,90


In [53]:
df.set_index(['name', 'exam'])

score
name   exam       
Alice  0        61
       1        70
       2        90
Bob    0        50
       1        80
       2        91
Freya  0        80
       1        90
       2        82
Merlin 0        90
       1       100
       2        92

One of the typical situation in which we find multi indexes are group by computations, in particular when we split
by multiple columns. For instance, when we computed numbers of passengers from different ports by ticket class in Titanic dataset what we got was a series with two-level row index.

In [54]:
# Data frame with two-level row index
titanic = sbn.load_dataset('titanic')

gdf = titanic.groupby(['class', 'embark_town'])['survived', 'fare'].mean()
gdf

survived        fare
class  embark_town                      
First  Cherbourg    0.694118  104.718529
       Queenstown   0.500000   90.000000
       Southampton  0.582677   70.364862
Second Cherbourg    0.529412   25.358335
       Queenstown   0.666667   12.350000
       Southampton  0.463415   20.327439
Third  Cherbourg    0.378788   11.214083
       Queenstown   0.375000   11.183393
       Southampton  0.189802   14.644083

We can also use indexes in group by. For this we specify `level` of the index that we will want to use to split our data.

In [56]:
gdf.groupby(level=1).mean()

,survived,fare
embark_town,,
Cherbourg,0.534106,47.096983
Queenstown,0.513889,37.844464
Southampton,0.411964,35.112128


In some sense, mulit indexes are just sequences of unique tuples.

In [57]:
gdf.index

MultiIndex([( 'First',   'Cherbourg'),
            ( 'First',  'Queenstown'),
            ( 'First', 'Southampton'),
            ('Second',   'Cherbourg'),
            ('Second',  'Queenstown'),
            ('Second', 'Southampton'),
            ( 'Third',   'Cherbourg'),
            ( 'Third',  'Queenstown'),
            ( 'Third', 'Southampton')],
           names=['class', 'embark_town'])

We can create `MultiIndex` objects by hand from sequences of tuples or form a cartesian product of multiple sequences of values.

In [58]:
# From list of tuples
pd.MultiIndex.from_tuples([
    ('a', 1),
    ('a', 2),
    ('b', 1),
    ('b', 2),
    ('b', 3)
])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2),
            ('b', 3)],
           )

In [59]:
# From cartesian product
pd.MultiIndex.from_product([
    ['a', 'b', 'c'], 
    [1, 2, 3, 4]
])

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('a', 4),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('b', 4),
            ('c', 1),
            ('c', 2),
            ('c', 3),
            ('c', 4)],
           )

### `MultiIndex` | Exercise 1.

You are provided with set of multiple measurements (5) for ten persons. They are arranged in 10 by 5 Numpy array.
Some value are missing (NaN). Your task is to arrange the data in a single `Series` object with index that differentiates
properly between persons and measurements.

Use your data structure to compute number of measurement per subject and number of subjects per measurements.

Finally, use your data structure (without changing it) to compute
average values by person and by measurement.

In [ ]:
X = np.where(np.random.uniform(0, 1, (10, 5)) < .1, np.nan, np.random.normal(100, 15, (10, 5)))
X

In [ ]:
# Your solution